In [1]:
import tensorflow as tf
import os
import json
from transformers import AutoTokenizer
import numpy as np
from mamba_model import ModelArgs, init_model

C:\Users\rabah\Documents\evry\M2 CNS\masteriale\SimpleLLM\Venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
def load_mamba_model(model_dir="mamba_model_sauvegarde"):
    # Charger la configuration
    with open(os.path.join(model_dir, "config.json"), "r") as f:
        config = json.load(f)

    # Charger le tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_dir)

    # Reconstruire le modèle
    args = ModelArgs(
        model_input_dims=config['model_input_dims'],
        model_states=config['model_states'],
        num_layers=config['num_layers'],
        dropout_rate=config['dropout_rate'],
        vocab_size=config['vocab_size'],
        num_classes=config['num_classes'],
        seq_length=config['seq_length'],
        conv_kernel_size=config['conv_kernel_size'],
        use_lm_head=config['use_lm_head'],
        loss=config['loss'],
    )

    model = init_model(args)

    # Charger les poids
    model.load_weights(os.path.join(model_dir, "model_weights.weights.h5"))

    return model, config, tokenizer

In [3]:
def generate_text(prompt, model, config, tokenizer, max_new_tokens=100,temperature=0.7, top_p=0.9, top_k=50):
    seq_length = config['seq_length']

    # Tokeniser le prompt
    prompt_tokens = tokenizer.encode(prompt)

    # Créer une séquence avec padding
    input_ids = np.array([prompt_tokens], dtype=np.int32)

    # Compléter avec du padding si nécessaire
    if input_ids.shape[1] < seq_length:
        padding = np.full((1, seq_length - input_ids.shape[1]), tokenizer.pad_token_id, dtype=np.int32)
        input_ids = np.concatenate([input_ids, padding], axis=1)
    else:
        input_ids = input_ids[:, :seq_length]

    generated_tokens = prompt_tokens.copy()

    for step in range(max_new_tokens):
        logits = model.predict(input_ids, verbose=0)

        next_logits = logits[0, -1, :]

        # Appliquer la temperature
        next_logits = next_logits / temperature

        # Softmax pour obtenir les probabilités
        probs = tf.nn.softmax(next_logits).numpy()

        # Top-k filtering
        if top_k > 0:
            top_k_indices = np.argsort(probs)[-top_k:]
            top_k_probs = probs[top_k_indices]
            top_k_probs = top_k_probs / top_k_probs.sum()

            next_token = np.random.choice(top_k_indices, p=top_k_probs)
        else:
            # Top-p (nucleus) sampling
            sorted_indices = np.argsort(probs)[::-1]
            sorted_probs = probs[sorted_indices]
            cumsum_probs = np.cumsum(sorted_probs)

            # Garder seulement les tokens dans le top-p
            valid_mask = cumsum_probs <= top_p
            if not np.any(valid_mask):
                valid_mask[0] = True

            valid_indices = sorted_indices[valid_mask]
            valid_probs = probs[valid_indices]
            valid_probs = valid_probs / valid_probs.sum()

            next_token = np.random.choice(valid_indices, p=valid_probs)

        generated_tokens.append(next_token)

        # Vérifier si on a généré un token de fin
        if next_token in [tokenizer.pad_token_id, tokenizer.sep_token_id, tokenizer.eos_token_id]:
            break

        # Mettre à jour input_ids (fenêtre glissante)
        next_input = np.array([[next_token]], dtype=np.int32)
        input_ids = np.concatenate([input_ids[:, 1:], next_input], axis=1)

        if (step + 1) % 20 == 0:
            print(f"  {step + 1}/{max_new_tokens} tokens générés...")

    # token vers texte
    generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    return generated_text

In [4]:
model, config, tokenizer = load_mamba_model("mamba_model_sauvegarde")

C:\Users\rabah\Documents\evry\M2 CNS\masteriale\SimpleLLM\Venv\Lib\site-packages\keras\src\layers\core\embedding.py:100: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
C:\Users\rabah\Documents\evry\M2 CNS\masteriale\SimpleLLM\Venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


C:\Users\rabah\Documents\evry\M2 CNS\masteriale\SimpleLLM\Venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 232 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [5]:
texte = generate_text(
    "Machine learning is",
    model, config, tokenizer,
    max_new_tokens=100,
    temperature=0.8
)
print(texte)

  20/100 tokens générés...
  40/100 tokens générés...
  60/100 tokens générés...
  80/100 tokens générés...
  100/100 tokens générés...
machine learning is set work in @ on "walk have of, on girl and un was " the before ier single of as into a 1965 record while that s performance @ doubt music a 1965 at she allmusic in of news who from was at set no felt 1965 byju. had by @ju a of were sun stage ] boys of - that ' the into itsno on was song ] by themselves images s only what as " for. us next album ross also london s a allmusic on "eti left of
